# 데이터 전처리: 이상치(Outlier)와 결측치(Missing Value)

## 1. 이상치(Outlier)

### 1.1. 이상치란?

이상치(Outlier)는 전체 데이터의 분포에서 크게 벗어난 값을 의미합니다. 다른 데이터 포인트들과 비교했을 때 비정상적으로 높거나 낮은 값을 가지는 데이터입니다.

### 1.2. 이상치 발생 원인

- **데이터 입력 오류**: 사람이 데이터를 입력하는 과정에서 발생한 실수
- **측정 오류**: 데이터 수집 장비의 오류나 잘못된 측정 방식으로 인한 오류
- **실험 오류**: 실험 환경의 통제 실패로 인한 비정상적인 데이터
- **의도적인 이상치**: 사기 탐지 등에서 비정상적인 패턴 자체가 분석 대상이 되는 경우

### 1.3. 이상치 처리 방법

이상치를 처리하지 않으면 데이터 분석 결과가 왜곡될 수 있습니다. 예를 들어, 데이터의 평균이 이상치 때문에 실제 분포를 대표하지 못하게 되거나, 머신러닝 모델의 성능이 저하될 수 있습니다.

1.  **삭제 (Deletion)**
    - 이상치를 포함하는 행(row) 전체를 제거합니다.
    - 가장 간단한 방법이지만, 데이터 손실이 발생할 수 있다는 단점이 있습니다. 특히 데이터 양이 적을 경우 신중하게 사용해야 합니다.

2.  **대체 (Replacement)**
    - 이상치를 다른 값으로 대체합니다.
    - 통계적으로 의미 있는 값(예: 중앙값, 평균값)이나, 논리적으로 가능한 최솟값/최댓값 등으로 대체할 수 있습니다.

3.  **변환 (Transformation)**
    - 데이터의 분포를 변경하여 이상치의 영향을 줄입니다.
    - 로그 변환(Log Transformation) 등을 사용하여 큰 값을 가진 데이터의 스케일을 줄일 수 있습니다.

### 1.4. 이상치 탐지 기법

#### 1) IQR (Interquartile Range) 방법
데이터의 사분위수를 이용하여 이상치를 탐지하는 가장 일반적인 방법입니다.

- **IQR**: Q3 (3사분위수) - Q1 (1사분위수)
- **정상 범위**: `Q1 - 1.5 * IQR` ~ `Q3 + 1.5 * IQR`
- 이 범위를 벗어나는 값을 이상치로 간주합니다.

In [ ]:
import pandas as pd
import numpy as np

# 예제 데이터프레임 생성
data = {
    'score': [78, 85, 92, 88, 76, 95, 81, 79, 150, 90, -20, 83]
}
df = pd.DataFrame(data)
print("Original DataFrame:")
print(df)

# IQR을 이용한 이상치 탐지
Q1 = df['score'].quantile(0.25)
Q3 = df['score'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\n1사분위수(Q1): {Q1}")
print(f"3사분위수(Q3): {Q3}")
print(f"IQR: {IQR}")
print(f"정상 범위: {lower_bound} ~ {upper_bound}")

# 이상치 확인
outliers = df[(df['score'] < lower_bound) | (df['score'] > upper_bound)]
print("\n--- 이상치 탐지 결과 ---")
print(outliers)

#### 2) 이상치 처리 예제

In [ ]:
# 방법 1: 이상치 삭제
df_no_outliers_deleted = df[(df['score'] >= lower_bound) & (df['score'] <= upper_bound)]
print("--- 이상치 삭제 후 DataFrame ---")
print(df_no_outliers_deleted)

# 방법 2: 이상치를 경계값으로 대체
# np.where(조건, 참일 때 값, 거짓일 때 값)
df_replaced = df.copy()
df_replaced['score'] = np.where(df_replaced['score'] > upper_bound, upper_bound, df_replaced['score'])
df_replaced['score'] = np.where(df_replaced['score'] < lower_bound, lower_bound, df_replaced['score'])

print("\n--- 이상치 대체 후 DataFrame ---")
print(df_replaced)

## 2. 결측치(Missing Value)

### 2.1. 결측치란?

결측치(Missing Value)는 데이터셋에 값이 존재하지 않는, 즉 비어있는 항목을 의미합니다. Pandas에서는 주로 `NaN` (Not a Number)으로 표시됩니다.

### 2.2. 결측치 발생 원인

- **정보 수집 실패**: 설문조사에서 응답자가 특정 질문에 답하지 않은 경우
- **데이터 입력 누락**: 데이터를 수집하거나 입력하는 과정에서 값이 누락된 경우
- **데이터 병합 오류**: 여러 데이터 소스를 병합할 때, 특정 데이터에 값이 없는 경우

### 2.3. 결측치 처리 방법

대부분의 머신러닝 알고리즘은 결측치가 있는 데이터를 처리하지 못하므로, 학습 전에 반드시 처리해야 합니다.

1.  **삭제 (Deletion)**
    - **행 삭제 (Listwise Deletion)**: 결측치가 하나라도 포함된 행 전체를 삭제합니다. 데이터 손실이 클 수 있습니다.
    - **열 삭제 (Column Deletion)**: 특정 열에 결측치가 너무 많을 경우 해당 열 자체를 삭제합니다. 변수(feature) 자체가 사라지는 단점이 있습니다.

2.  **대체 (Imputation)**
    - 결측치를 다른 값으로 채우는 방법입니다. 데이터의 특성을 고려하여 적절한 값을 선택해야 합니다.
    - **특정 값으로 대체**: `0`, `'Unknown'` 등 의미를 부여할 수 있는 특정 값으로 채웁니다.
    - **통계 값으로 대체**: 
        - **평균(Mean)**: 데이터가 정규분포에 가까울 때 사용합니다. 이상치에 민감합니다.
        - **중앙값(Median)**: 데이터 분포가 한쪽으로 치우쳐(skewed) 있을 때 사용합니다. 이상치에 덜 민감합니다.
        - **최빈값(Mode)**: 범주형(categorical) 데이터에 주로 사용합니다.
    - **이전/이후 값으로 대체**: 시계열 데이터처럼 데이터의 순서가 의미 있을 때, 바로 앞(`ffill`)이나 뒤(`bfill`)의 값으로 채웁니다.

In [ ]:
import pandas as pd
import numpy as np

# 예제 데이터프레임 생성 (결측치 포함)
data = {
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Emily', 'Frank'],
    'age': [25, 30, np.nan, 22, 35, np.nan],
    'salary': [50000, 80000, 65000, np.nan, 90000, 45000],
    'department': ['HR', 'Engineering', 'Marketing', 'HR', np.nan, 'Engineering']
}
df_missing = pd.DataFrame(data)

print("Original DataFrame with Missing Values:")
print(df_missing)

#### 1) 결측치 확인

In [ ]:
# isnull() 또는 isna()를 사용하여 결측치 확인 (True: 결측치)
print("\n--- 결측치 확인 (isnull) ---")
print(df_missing.isnull())

# 결측치 개수 확인
print("\n--- 컬럼별 결측치 개수 ---")
print(df_missing.isnull().sum())

# 데이터프레임 정보 확인
print("\n--- DataFrame 정보 (info) ---")
df_missing.info()

#### 2) 결측치 처리 예제

In [ ]:
# 방법 1: 결측치가 있는 행 삭제 (dropna)
df_deleted_rows = df_missing.dropna(axis=0) # axis=0은 행을 의미
print("--- 결측치 행 삭제 후 ---")
print(df_deleted_rows)

# 방법 2: 특정 열에 결측치가 너무 많을 경우 열 삭제
# 예시로 salary 열을 삭제 (실제로는 결측치 비율을 보고 결정)
df_deleted_cols = df_missing.drop('salary', axis=1) # axis=1은 열을 의미
print("\n--- 'salary' 열 삭제 후 ---")
print(df_deleted_cols)

In [ ]:
# 방법 3: 결측치 대체 (fillna)
df_filled = df_missing.copy()

# 3-1) 특정 값으로 대체 (age는 0으로, department는 'Unknown'으로)
df_filled_specific = df_missing.copy()
df_filled_specific['age'] = df_filled_specific['age'].fillna(0)
df_filled_specific['department'] = df_filled_specific['department'].fillna('Unknown')
print("--- 특정 값으로 대체 후 ---")
print(df_filled_specific)

# 3-2) 통계 값으로 대체
df_filled_stats = df_missing.copy()

# age는 중앙값(median)으로 대체 (이상치에 덜 민감)
age_median = df_filled_stats['age'].median()
df_filled_stats['age'].fillna(age_median, inplace=True)

# salary는 평균(mean)으로 대체
salary_mean = df_filled_stats['salary'].mean()
df_filled_stats['salary'].fillna(salary_mean, inplace=True)

# department는 최빈값(mode)으로 대체
dept_mode = df_filled_stats['department'].mode()[0] # mode()는 Series를 반환하므로 [0]으로 값 선택
df_filled_stats['department'].fillna(dept_mode, inplace=True)

print("\n--- 통계 값으로 대체 후 ---")
print(df_filled_stats)